In [ ]:
import csv
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
import statistics
from scipy.signal import savgol_filter
from scipy.signal import find_peaks

import importlib
import TeloFast5V2
from TeloFast5V2 import *
# from multiprocessing import Pool
# !pip install pandarallel

from pandarallel import pandarallel

importlib.reload(TeloFast5V2)


In [ ]:
file_path = './data/F63Fast5/SquigglePull.F63.batch0-1-2.tsv.gz'

# Read the first line of the TSV file using csv.reader
# with open(file_path, 'r', newline='') as tsv_file:
#     tsv_reader = csv.reader(tsv_file, delimiter='\t')
#     header = next(tsv_reader)

# signal = header[2:]
# signal = [float(i) for i in signal]

# ****** Randomize the inputted table before grabbing subset, the first reads will be shorter than those read later on 

reducedDataDf = pd.read_csv(file_path, sep='\t', compression="gzip", error_bad_lines=False)

reducedDataDf["signal"] = reducedDataDf["signal"].apply(lambda x: [float(i) for i in ast.literal_eval(x)])
signalDf = reducedDataDf

In [ ]:
print("Start processing")
dfDvsG = pd.read_csv("../Data/F63/reads.table_F63.TeloBP.prim.ClaI.NB50uq_alnScTh.20.txt", sep="\t")
print("Finished reading dfDvsG")
file_path = './data/F63Fast5/SquigglePull.F63.batch0-1-2.tsv/SquigglePull.data.1000SubSample.tsv'

In [ ]:
# print out the head of file_path
with open(file_path, 'r', newline='') as tsv_file:
    tsv_reader = csv.reader(tsv_file, delimiter='\t')
    # grab the 7th line
    for i in range(7):
        header = next(tsv_reader)
    print(header)
    print(len(header))

In [ ]:
with open(file_path, 'r', newline='') as tsv_file:
    chunk = csv.reader(tsv_file, delimiter='\t')

    print("Processing chunk")
    # for each row in chunk, convert signal column from string to list of floats
    # for index, row in chunk.iterrows():
    for row in chunk:
        qnm = row[1]
        print(qnm)
        # signal = row[2:].tolist()
        signal = row[2:]

        # print(signal)
        # remove nan and non numerics
        signal = [float(x) for x in signal if isinstance(x, (int, float)) or (isinstance(x, str) and x != 'nan' and not x.endswith('.fast5') and "-" not in x)]
        # signal = [x for x in signal if str(x) != 'nan']
        # signal = [float(i) for i in signal]
        chunkList.append([qnm, signal])
    print(len(chunkList))
    # break close file
    tsv_file.close()

print("done loading")
# chunkList = chunkList[0:100]

# # chunkList = pd.read_csv(file_path, sep='\t', error_bad_lines=False)

print(len(chunkList))
signalDf = pd.DataFrame(chunkList, columns=["qname", "signal"])


# chunk["signal"] = chunk["signal"].apply(lambda x: [float(i) for i in ast.literal_eval(x)])
# signalDf = chunk
# drop all columns in signalDf except qname and signal
signalDf = signalDf[['qname', 'signal']] 
dvsGWithSignal = dfDvsG[dfDvsG["qname"].isin(signalDf["qname"])]    
dvsGWithSignal["isGStrand"] = dvsGWithSignal.apply(lambda row: isGStrand(row["chr"][-1] ,row["strand"]),axis=1)
signalDf = signalDf.merge(dvsGWithSignal[['qname', 'isGStrand']], on='qname', how='left')

pandarallel.initialize(progress_bar=True )

signalDf["PeakCountBasedLengthGuppy"] = signalDf.parallel_apply(getTeloCountLength,axis=1)

# drop signal column
saveDf = signalDf.drop(columns=["signal"])
# chunks.append(saveDf)

# outputDf = pd.concat(chunks, axis=0)
dvsGWithSignalShortened = signalDf
saveDf.to_csv('output/F63.subset1000.peak.count.V3.1.tsv', sep='\t')
print("Finished processing")

In [ ]:
# dfDvsG = pd.read_csv("./data/F61.F71.telomeres.for.Kayarash.txt", sep="\t")
dfDvsG = pd.read_csv("../Data/F63/reads.table_F63.TeloBP.prim.ClaI.NB50uq_alnScTh.20.txt", sep="\t")

# dfDvsG = pd.read_csv("./data/mappings/for.Ramin.Tel.diff.by.mapping.10000.txt", sep=" ")
dfDvsG
dvsGWithSignal = dfDvsG[dfDvsG["qname"].isin(signalDf["qname"])]    